In [ ]:
import os, random, glob, sys, subprocess, time, scipy, png, cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images, imsave
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from osgeo import gdal, gdal_array

import tensorflow as tf
import tensorflow.keras as tfkeras

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, ZeroPadding2D
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, Adamax, Adadelta, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import contextlib

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
#FOR TRAINING IMAGES AND SELF MADE MASKS
import os
from osgeo import gdal


directory = r'U:\Training_Data\Training_Images\Test\T54SVF_20190314'
os.chdir(directory)

width = 5490
height = 5490
tilesize = 549

for i in range(0, width, tilesize):
    for j in range(0, height, tilesize):
        gdal.Translate(destName = str(i)+'_'+str(j)+'_T54SVF.png',
                      srcDS = 'T54SVF_20190314.tif',
                      bandList = [9,4,3,2],
                      srcWin = [i,j,tilesize,tilesize],
                      noData = None,
                      format = "PNG"
                      )

In [ ]:
#append all filenames in a given directory
import os
 
path = r'U:\Training_Data\Training_Images\Test\T44TLM_20190418\Raster_Mask'
for filename in os.listdir(path):
    filename_without_ext = os.path.splitext(filename)[0]
    extension = os.path.splitext(filename)[1]
    new_file_name = filename_without_ext+"_T44TLM" #<--- this will be appended to the end of the filename
    new_file_name_with_ext = new_file_name+extension
    #print(new_file_name_with_ext)
    os.rename(os.path.join(path,filename),os.path.join(path,new_file_name_with_ext))

In [ ]:
# Set some parameters
im_width = 512
im_height = 512
border = 0

In [ ]:
train_path = r'U:\Training_Data\Training_Images\Test\Unet_Train'
ids = next(os.walk(train_path))[2] # list of names all images in the given path
for i in ids:
    if '.xml' in i:
        ids.remove(i)

try:
    ids.remove('Thumbs.db')
except:
    pass

print("No. of images = ", len(ids))
print(ids)

In [ ]:
X = np.zeros((len(ids), im_height, im_width, 4), dtype=np.float32)
y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

In [ ]:
# tqdm is used to display the progress bar
import os
os.chdir(r'U:\Training_Data\Training_Images\Test\Unet_Train')
for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
    # Load images
    #x_img = gdal_array.LoadFile(id_)
    img = load_img(id_)
    x_img = img_to_array(img)
    x_img = resize(x_img, (im_width, im_height, 4), mode = 'edge', preserve_range = True)
    # Load masks
    mask = img_to_array(load_img("Raster_Masks/Split/"+id_,color_mode='grayscale'))
    mask = resize(mask, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
    # Save images
    X[n] = x_img/255.0
    y[n] = mask/255.0

In [ ]:
# Visualize any randome image along with the mask
ix = random.randint(0, len(X))
has_mask = y[ix].max() > 0 # cloud indicator

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 15))

ax1.imshow(X[ix, ..., 0], cmap = 'gray', interpolation = 'bilinear')
if has_mask: # if salt
    # draw a boundary(contour) in the original image separating cloud and non-cloud areas
    ax1.contour(y[ix].squeeze(), colors = 'k', linewidths = 5, levels = [0.5])
ax1.set_title('Image')

ax2.imshow(y[ix].squeeze(), cmap = 'gray', interpolation = 'bilinear')
ax2.set_title('Mask')

In [ ]:
def conv2d_block4(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #third layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #fourth layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def conv2d_block5(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #third layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #fourth layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #fifth layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x
  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
# Encoder
    c1 = conv2d_block5(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block5(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block5(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block4(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block4(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)

    # Decoder
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block4(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block5(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block5(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block5(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
tfkeras.backend.clear_session()

In [ ]:
input_img = Input((im_height, im_width, 4), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.1, batchnorm=True)
model.compile(optimizer=RMSprop(lr=0.001), loss='poisson', metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
os.chdir(r'U:\Training_Data\Training_Images\Test')
from contextlib import redirect_stdout

with open('modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()


In [ ]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000001, verbose=1),
    ModelCheckpoint(r'U:\Training_Data\CNN_Models\Model_116.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
t3 = time.time()
results = model.fit(X, y, batch_size=1, epochs=4,validation_split=0.1,shuffle=True)
t4 = time.time()
Total = round((t4-t3)/60, 1)
print('Training complete, took '+str(Total)+' minutes.')

In [ ]:
train_path = r'U:\Training_Data\Training_Images\Test\Unet_Test'
ids = next(os.walk(train_path))[2] # list of names all images in the given path
for i in ids:
    if '.xml' in i:
        ids.remove(i)

try:
    ids.remove('Thumbs.db')
except:
    pass

print("No. of images = ", len(ids))

In [ ]:
fixed_list = [0,9,1,2,3,4,5,6,7,8,10,19,11,12,13,14,15,16,17,18,20,29,21,22,23,24,25,26,27,28,
             30,39,31,32,33,34,35,36,37,38,40,49,41,42,43,44,45,46,47,48,50,59,51,52,53,54,55,56,57,58,
             60,69,61,62,63,64,65,66,67,68,70,79,71,72,73,74,75,76,77,78,80,89,81,82,83,84,85,86,87,88,
             90,99,91,92,93,94,95,96,97,98]

new_ids = [ids[i] for i in fixed_list]

In [ ]:
X_test = np.zeros((len(ids), im_height, im_width, 4), dtype=np.float32)

In [ ]:
# tqdm is used to display the progress bar
import os
os.chdir(r'U:\Training_Data\Training_Images\Test\Unet_Test')
for n, id_ in tqdm_notebook(enumerate(new_ids), total=len(new_ids)):
    # Load images
    img = load_img(id_)
    x_img = img_to_array(img)
    #x_img = gdal_array.LoadFile(id_)
    x_img = resize(x_img, (im_height, im_width, 4), mode = 'edge', preserve_range = True)
    X_test[n] = x_img/255.0

In [ ]:
model = load_model(r'U:\Training_Data\CNN_Models\Model_117_whole.h5')

In [ ]:
model.save(r'U:\Training_Data\CNN_Models\Model_117_whole.h5')

In [ ]:
t3 = time.time()
result = model.predict(X_test,batch_size=4)
t4 = time.time()
Total = round((t4-t3)/60, 1)
print('Predictions complete, took '+str(Total)+' minutes.')

In [ ]:
# Visualize any randome image along with the mask
ix = random.randint(0, 99)
print(ix)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 15))

ax1.imshow(X_test[ix, ..., 0], cmap = 'gray', interpolation = 'bilinear')
ax1.set_title('Image')

ax2.imshow(result[ix].squeeze(), cmap = 'gray', interpolation = 'nearest')
ax2.set_title('Mask')

In [ ]:
result_new = result*10000

In [ ]:
result[0][0]

In [ ]:
from PIL import Image
os.chdir(r'U:\Training_Data\Training_Images\Test\Unet_Test\Result_Mask')
height = 5490
width = 5490
tilesize = 549

for i,k in enumerate(new_ids):
    x_img = cv2.resize(result_new[i].astype('float32'), (549, 549))
    cv2.imwrite(str(k),x_img)

In [ ]:
os.chdir(r'U:\Training_Data\Training_Images\Test\Unet_Test\Result_Mask')

img_list = glob.glob('*.png')

gdal.Warp(destNameOrDestDS='test.tif',
          srcDSOrSrcDSTab=img_list,
          resampleAlg='near',
          width=5490,
          height=5490,
          format='GTiff',
          outputType=gdal.GDT_Byte
          )

In [ ]:
#USE THIS FOR RASTER CALCULATIONS

gdal_path = r'C:\Users\jbrown\AppData\Local\conda\conda\envs\gpu\Scripts'
gdal_calc_path = os.path.join(gdal_path, 'gdal_calc.py')

# Arguements.
input_file_path = 'test.tif'
output_file_path = 'test_filtered.tif'
calc_expr = '"A>=5.0"'
nodata = '0'
typeof = '"Byte"'

# Generate string of process.
gdal_calc_str = 'python {0} -A {1} --outfile={2} --calc={3} --NoDataValue={4} --type={5}'
gdal_calc_process = gdal_calc_str.format(gdal_calc_path, input_file_path, 
    output_file_path, calc_expr, nodata, typeof)

# Call process.
os.system(gdal_calc_process)

In [ ]:
os.chdir(r'U:\Training_Data\Training_Images\Test\Unet_Test\Result_Mask')
sys.path.append(r'C:\Users\jbrown\AppData\Local\conda\conda\envs\gpu\Scripts')
gm = os.path.join('C:\\','Users','jbrown','AppData','Local','conda','conda','envs','neural2','Scripts','gdal_sieve.py')
sieve_command = ["python", gm,'-st','10','-8','-nomask','-of','GTiff','test_filtered.tif','_sieved.tif']
subprocess.call(sieve_command,shell=True)

gm = os.path.join('C:\\','Users','jbrown','AppData','Local','conda','conda','envs','gpu','Scripts','gdal_proximity.py')
prox_command = ["python", gm, '-of','GTiff','-ot','Byte','-distunits','PIXEL','-maxdist','8','-ot','Byte','-fixed-buf-val','1.0','_sieved.tif','_prox.tif']
subprocess.call(prox_command,shell=True)


sys.path.append(r'C:\Users\jbrown\AppData\Local\conda\conda\envs\gpu\Scripts')
gm = os.path.join('C:\\','Users','jbrown','AppData','Local','conda','conda','envs','gpu','Scripts','gdal_calc.py')
calc_command = ["python", gm,'--calc','A+B','-A','_prox.tif','-B','_sieved.tif','--NoDataValue','255','--outfile','final_test.tif',]
subprocess.call(calc_command,shell=True)
